In [1]:
using LinearAlgebra
using Statistics # for mean
# using Roots
using NLsolve

In [2]:
function polybas(kmin,kmax,Np,kgrid)
    
    # Np: 多項式の次数-1
    # Ng: グリッドの数
    Ng = size(kgrid,1)
    x = (2/(kmax-kmin))*(kgrid .- kmin) .- 1
    
    # 基底関数の行列(NgxNp)を再帰的に求める
    T = zeros(Ng,Np)
    T0 = ones(Ng)
    T1 = x
    T2 = 2*x.*T1 - T0
    T[:,1] = T1
    T[:,2] = T2
    
    for i=3:Np-1
        T[:,i] = 2*x.*T[:,i-1] - T[:,i-2] 
    end
    
    T = [T0 T[:,1:(Np-1)]]
    
    return T
    
end

polybas (generic function with 1 method)

In [3]:
function polygrid(kmin,kmax,N)
    
    temp = collect(LinRange(0,N-1,N))
    x = -cos.((pi/(N-1))*temp) # チェビシェフ極値点
    #temp = collect(LinRange(1,N-1,N-1))
    #x = [0; -cos((pi/2/(N-1))*(2*temp .- 1))] # チェビシェフゼロ点
    
    # xからkに変換
    k = 0.5*(kmax-kmin)*(x .+ 1) .+ kmin
   
    return k
    
end

polygrid (generic function with 1 method)

In [4]:
struct Model{TI<:Integer, TF<:AbstractFloat, TV<:Vector}
    β::TF       # 割引因子
    γ::TF       # 相対的危険回避度
    α::TF       # 資本分配率
    δ::TF       # 固定資本減耗      
    # 定常状態の値
    ykss::TF
    kss::TF
    yss::TF
    css::TF
    nk::TI      # 資本グリッドの個数
    kmax::TF    # 資本グリッドの最大値
    kmin::TF    # 資本グリッドの最小値 (0にすると生産が出来なくなる)
    kgrid::TV   # 資本グリッド
    T::Array{Float64,2}
    invT::Array{Float64,2}
    maxiter::TI # 繰り返し計算の最大値
    tol::TF     # 許容誤差
end

In [5]:
function EulerEq_cheb(x,m,capital,theta)
# cを与えたときのオイラー方程式の残差を返す関数
    
    cons = x[1]
    wealth = capital^m.α + (1-m.δ)*capital
    
    kprime = wealth - cons
    # トリック: k'は正の値しか取らない
    kprime = max(m.kgrid[1],kprime)

    T = polybas(m.kmin,m.kmax,m.nk,[kprime]) # the type of [kprime] is Array{Float64,1}
    temp = T*theta
    cnext = temp[1]
    # オイラー方程式
    res = mu_CRRA(cons,m.γ) - m.β*mu_CRRA(cnext,m.γ)*(m.α*kprime.^(m.α-1) + (1-m.δ))

    return res 
end

EulerEq_cheb (generic function with 1 method)

In [6]:
function CRRA(cons::Real, gamma)
    """
    Compute CRRA utility function
    
    # Arguments

    - `cons::Real`: consumption value
    - `gamma::Real`: relative risk aversion
    
    # Return 
    - `util::Real`: utility value 
    """
    if gamma != 1.0
        util = cons^(1.0 - gamma) / (1.0 - gamma)
    else
        util = log(cons) 
    end
    return util
end

CRRA (generic function with 1 method)

In [7]:
function mu_CRRA(cons::Real, gamma)
    """
    Compute marginal utility of CRRA-type function
    
    # Arguments 
    - "cons::VecOrMat": consumption value
    - "gamma::Real": relative risk aversion
    
    # Return
    - "mu::Real": marginal utility 
    """
    
    mu = cons^-gamma
    return mu
end

mu_CRRA (generic function with 1 method)

In [8]:
function nti_cheb(m::Model)
    
    # *** 収束の基準 ***
    it = 1         # ループ・カウンター
    dif2 = 1.0     # 政策関数の繰り返し誤差
    tolfun = 1e-10 # NLsolveのオプション(最適化の許容誤差)

#     println(" ")
#     println("-+- Solve a neoclassical growth model with time iteration -+-")
#     println(" ")    
    
    ## STEP 1(b): 政策関数の初期値を当て推量    
    # 解析解 (for k'=g(k))
    p_true = m.β*m.α*(m.kgrid.^m.α)
    
    # 政策関数の初期化
    cfcn0 = m.kgrid
    #cfcn0 = m.css/m.kss*m.kgrid
    cfcn1 = zeros(m.nk)
        
    # 繰り返し誤差を保存する変数を設定 
    dif = zeros(2,m.maxiter)
    
    ## STEP 4: 価値関数を繰り返し計算
    while (it < m.maxiter && dif2 > m.tol)
        
        # 次期の政策関数を補間
        # 次期の政策関数を線形補間: m.nk=21のときは政策関数の形がおかしい???
        #cnext = Spline1D(m.kgrid,cfcn0,k=1,bc="extrapolate") #線形補間
        # 次期の価値関数をスプライン補間
        #cnext = Spline1D(m.kgrid,cfcn0,k=3,bc="extrapolate") #スプライン補間
        theta = m.invT*cfcn0
#         cfcn1 = zeros(m.nk)
        
        for i = 1:m.nk
            
            capital = m.kgrid[i]
            wealth = capital.^m.α + (1-m.δ).*capital
            
            # Juliaの最適化関数(NLsolve or Roots)を使って各グリッド上の政策関数の値を探す
            EulerEq_cheb!(x) = EulerEq_cheb(x,m,capital,theta) # x is a vector (for NLsolve)
#             EulerEq_cheb!(cons) = EulerEq_cheb([cons],m,capital,theta) # cons is a scalar (for Roots)
            # 最適化の初期値は古い政策関数の値
            # for Roots
#             cons = find_zero(EulerEq_cheb!,cfcn0[i])            
#             cfcn1[i] = cons
            # for NLsolve
            sol = nlsolve(EulerEq_cheb!,[cfcn0[i]],ftol=tolfun)
            # 最適化の初期値は定常状態の値: これでは解けない
#             sol = nlsolve(EulerEq_cheb!,[m.css],ftol=tolfun)
            cfcn1[i] = sol.zero[1]
            # グリッドごとに最適化の結果を確認
            #disp([cons capital wealth kprime]);
            #pause

        end
        
        # 繰り返し計算誤差を確認
        dif2 = maximum(abs.(cfcn1-cfcn0)) # 政策関数の繰り返し計算誤差(図示のため)
        
#         println([it dif2])
        
        # 収束途中の繰り返し計算誤差を保存
        dif[2,it] = dif2
        
        # 政策関数をアップデート
        cfcn0 = copy(cfcn1)
        
        it = it + 1
        
    end
        
    return cfcn0, dif 
end

nti_cheb (generic function with 1 method)

In [9]:
function calcerr(m,cfcn0)
## オイラー方程式から誤差を測定
    # 元のグリッドではオイラー方程式の誤差はゼロになるため、グリッドを細かくとる
    theta = m.invT*cfcn0
    kgrid_err = collect(LinRange(m.kmin,m.kmax,(m.nk-1)*10+1))
    T = polybas(m.kmin,m.kmax,m.nk,kgrid_err)
    cons = T*theta
    #cons_interp = Spline1D(m.kgrid,cfcn0,k=1,bc="extrapolate")
    #cons = cons_interp(kgrid_err)
    LHS  = mu_CRRA.(cons,m.γ)

    kp   = kgrid_err.^m.α + (1-m.δ)*kgrid_err - cons
    T = polybas(m.kmin,m.kmax,m.nk,kp)
    cnext = T*theta
    #cnext = cons_interp(kp)
    rent = m.α.*kp.^(m.α-1.0) .- m.δ
    RHS  = m.β.*(1 .+ rent).*mu_CRRA.(cnext,m.γ)

    err  = RHS./LHS.-1.0
    
    return err
    
end

calcerr (generic function with 1 method)

In [10]:
# カリブレーション
β = 0.96 # 割引因子
γ = 1.0  # 相対的危険回避度(異時点間の代替の弾力性の逆数)
α = 0.40 # 資本分配率
δ = 1.0  # 固定資本減耗(delta=1.0のときは解析解が存在)

# 定常状態の値
ykss = (1/β-1+δ)/α
kss = ykss^(1/(α-1))
yss = ykss*kss
css = yss-δ*kss

# kmax = 0.5  # 資本グリッドの最大値
# kmin = 0.05 # 資本グリッドの最小値 (0にすると生産が出来なくなる)
kmax = 1.2*kss # 資本グリッドの最大値
kmin = 0.8*kss # 資本グリッドの最小値 (0にすると生産が出来なくなる)

maxiter = 1000 # 繰り返し回数の最大値
tol = 1.0e-8;  # 許容誤差(STEP 2)

In [17]:
norms = zeros(3,2)
times = zeros(3,2)

nkvec = [3 5 9]';

for i=1:3

    ## STEP 1(a): グリッド生成
    nk = nkvec[i]     # グリッドの数
    kgrid = polygrid(kmin,kmax,nk)
    T = polybas(kmin,kmax,nk,kgrid)
    invT = inv(T)
    m = Model(β,γ,α,δ,ykss,kss,yss,css,nk,kmax,kmin,kgrid,T,invT,maxiter,tol)

    # time iteration
#     @time cfcn0, dif = nti(m)
    times[i,1] = @elapsed cfcn0, dif = nti_cheb(m) # different from @time???
#     tic();
#     cfcn0, dif = nti(m) # different from @time???
#     times[i,1] = toc();

    err = calcerr(m,cfcn0)
    norms[i,:] = log10.([mean(abs.(err)) maximum(abs.(err))])
    
end

times[:,2] = times[:,1]./times[1,1]

println(norms)
println(times)

[-3.49855 -3.23498; -5.79966 -5.48986; -7.67968 -7.6768]
[4.00402 1.0; 0.00145554 0.00036352; 0.00441488 0.00110261]


In [12]:
# ## 最終的な政策関数が得られてから貯蓄関数を計算
# pfcn0 = m.kgrid.^(m.α) + (1-m.δ).*m.kgrid - cfcn0;

In [13]:
# ## 解析的解
# p_true = m.β*m.α*(m.kgrid.^m.α);

In [14]:
# plot(m.kgrid,pfcn0)
# plot!(m.kgrid,p_true)
# plot!(m.kgrid,m.kgrid)

In [15]:
#err2 = readdlm("err_ndp.csv")
#err2 = vec(err2);

In [16]:
# plot(kgrid_err,abs.(err))
#plot!(kgrid_err,abs.(err2))